# Learning to Decode 7T-like MR Image Reconstruction from 3T MR Images:   Part-2

In this part you will be using the weights that were trained in Part-1 and will predict on the unseen trained data. You'll also learn how to finally save the 2D images as a combined volume.<br>

**Note**: You would probably be interested in reading the paper, if so, you can find the paper <a href=" https://arxiv.org/abs/1806.06886" target="_blank">here</a>. 

In a nutshell, you'll address the following topics:

<ul>
    <li>You will start off by importing the required <a href='#modules'>modules</a> that you will need in order to train your deep learning model,</li>
<li>Then you will be briefed about <a href='#data'>3T and 7T</a> MRI dataset,
<li>Then you'll define the <a href='#initialisers'>intializers</a> and <a href='#load'>load</a> the 3T and 7T test dataset, while loading the data you will also resize the images on the fly,</li>
<li>Next you will <a href='#preprocess'>preprocess</a> the loaded data: Convert the train and test lists into numpy matrices, convert the matrices type into float32, rescale the matrices using max-min strategy, reshape the arrays and finally split the data into 80% training and remaining 20% into validation set,</li>
<li>Then, you'll create the <a href='#model'>1-Encoder-3-Decoder</a> Architecture: comprising of merge connections and multi-decoders,</li>
<li>Next, you'll <a href='#loss'>define</a>: loss function, three different models and finally load the trained weights,</li>
<li>Finally, its time that you will <a href='#test'>predict</a> using your merge and multi-decoder model on the unseen data and save both the quantitative as well as qualitative results. You'll also learn how to save 2D images as a combined volume using `nibabel`.  </li>
</ul>    

# Testing Script

### Python Module Dependencies

Before you start following along make sure you have the exact same module versions as stated below:

<ul>
    <li>Keras==2.0.4</li>
    <li>tensorflow==1.8.0</li>
    <li>scipy==0.19.0</li>
    <li>numpy==1.14.5</li>
    <li>Pillow==4.1.1</li>
    <li>nibabel==2.1.0</li>
    <li>scikit_learn==0.18.1</li>
    </ul>

**Note** : Before you begin, please note that the model will be trained on a system with Nvidia 1080 Ti GPU Xeon e5 GeForce processor with 32GB RAM. If you are using Jupyter Notebook, you will need to add three more lines of code where you specify CUDA device order and CUDA visible devices using a module called os.

In the code below, you basically set environment variables in the notebook using os.environ. It's good to do the following before initializing Keras to limit Keras backend TensorFlow to use the first GPU. If the machine on which you train on has a GPU on 0, make sure to use 0 instead of 1. You can check that by running a simple command on your terminal: for example, nvidia-smi

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0" #model will be trained on GPU 

## Importing the modules

First, you import all the required modules like tensorflow, numpy and most importantly keras and the required functions or layers like `Input`, `Conv2D`,`MaxPooling2D` etc. since you'll be using all these frameworks for training the model!<br>
In order to read the nifti format images, you also have to import a module called `nibabel`.

In [2]:
import os
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
import numpy as np
import scipy.misc
import numpy.random as rng
from sklearn.utils import shuffle
import nibabel as nib
from sklearn.cross_validation import train_test_split
import math

<a id= 'data'> </a>
## Understanding the Brain MRI 3T and 7T Dataset

The brain MRI 3T and 7T dataset consists of 3D volumes each volume has in total 207 slices/images of brain MRI's taken at different slices of the brain. Each slice is of dimension 173 x 173. The images are single channel grayscale images. There are in total 39 subjects, each subject containing the MRI scan of a patient. The image format is not jpeg,png etc. but rather nifti format. You will see in later section how to read the nifti format images.

The dataset consists of T1 modality MR images, T1 sequences are traditionally considered good for evaluation of anatomic structures. The dataset on which you will be working today consists of 3T  and 7T Brain MRI's. 

The dataset is public and is available for download at this <a href=" https://www.humanconnectome.org/study/hcp-young-adult/document/1200-subjects-data-release" target="_blank">source</a>. 

<a id= 'initialisers'> </a>
## Defining the Initializers

Let's first define the dimensions of the data, we will resize the image dimension from 173x173 to 176x176 in the data reading part. Here we also define the data directory, batch size we use for training the model, number of channels, an `Input()` layer,train and test matrices as a list and finally for rescaling we load the text file which has the minimum and maximum values of the MRI dataset.

In [3]:
x,y = 173,173
full_z = 207
resizeTo=176
inChannel = outChannel = 1
input_shape=(x,y,inChannel)
input_img = Input(shape = (resizeTo, resizeTo, inChannel))

In [ ]:
train_matrix = []
test_matrix = []
ff = os.listdir("../test_crossval1")
save = "../Result_nii_crossval1/"
folder_ground = os.listdir("../test_g_crossval1")
ToPredict_images=[]
predict_matrix=[]

In [ ]:
ground_images=[]
ground_matrix=[]
min_max=np.loadtxt('../maxANDmin.txt')

<a id= 'load'> </a>
## Load the Testing Volumes

Next we load the mri data using `nibabel` library and resize the images from 173 x 173 to 176 x 176 by padding zeros in x and y dimension.

Note that when you load a Nifti format volume, Nibabel does not load the image array. It waits until you ask for the array data. The normal way to ask for the array data is to call the get_data() method.

Since you want the 2D slices instead of 3D, you will use the `train and test` lists that you initialised before; every time you read a volume, you will iterate over all the complete 207 slices of the 3D volume and append each slice one by one in to a list.



In [ ]:
for f in ff:
    temp = np.zeros([resizeTo,full_z,resizeTo])
    a = nib.load("../test_crossval1/" + f)
    affine = a.affine
    a = a.get_data()
    temp[3:,:,3:] = a
    a = temp
    for j in range(full_z):
        predict_matrix.append(a[:,j,:])

In [ ]:
for f in ff:
    temp = np.zeros([resizeTo,full_z,resizeTo])
    a = nib.load("../test_g_crossval1/" + f)
    affine = a.affine
    a = a.get_data()
    temp[3:,:,3:] = a
    a = temp
    for j in range(full_z):
        ground_matrix.append(a[:,j,:])

<a id= 'preprocess'> </a>
## Data Preprocessing

Since 3t and 7T test matrices is a list you will use numpy module to convert the list in to a numpy array.<br>
<br>
Further you will convert the `type` of the numpy array as `float32` and also rescale both the `input` and the `ground truth`.

In [ ]:
ToPredict_images = np.asarray(predict_matrix)
ToPredict_images = ToPredict_images.astype('float32')
mx = min_max[0]
mn = min_max[1]
ToPredict_images[:,:,:,0] = (ToPredict_images[:,:,:,0] - mn ) / (mx - mn)

In [ ]:
ground_images = np.asarray(ground_matrix)
ground_images = ground_images.astype('float32')
ground_images[:,:,:,0] = (ground_images[:,:,:,0] - mn ) / (mx - mn)

Next you will create two new variables `ToPredict_images`(3T Test/input) and `ground_images` (7T Test/ground truth) of the shape train and test matrix. This will be a 4D matrix in which the first dimension will be the total number of images, second and third being the dimension of each image and last dimension being the number of channels which is one in this case."

In [ ]:
ToPredict_images=np.zeros(shape=[(ToPredict_images.shape[0]),(ToPredict_images.shape[1]),(ToPredict_images.shape[2]),(1)])
ground_images=np.zeros(shape=[(ground_images.shape[0]),(ground_images.shape[1]),(ground_images.shape[2]),(1)])

Then you will iterate over all the images one by one, each time you will reshape the train and test matrix to 176 x 176 and append it to `ToPredict_images`(3T Test/input) and `ground_images` (7T Test/ground truth) respectively.

In [ ]:
for i in range(ToPredict_images.shape[0]):
    ToPredict_images[i,:,:,0] = ToPredict_images[i,:,:].reshape(resizeTo,resizeTo)

In [ ]:
for i in range(ground_images.shape[0]):
    ground_images[i,:,:,0] = ground_images[i,:,:].reshape(resizeTo,resizeTo)

<a id= 'model'> </a>
## The Model!

<div style="text-align:center"><img src="miccai1.jpg"></div><br>
<div style="text-align:center">Figure: Architecture of the Model</div><br>
<div style="text-align:center">Image taken from this  <a href="https://arxiv.org/abs/1806.06886" target="_blank">paper</a>.  </div>

### Encoder

In [4]:
def encoder(input_img):
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3,3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3,3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, (3, 3), activation='sigmoid', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    return conv5,conv4,conv3,conv2,conv1

### Decoder

In [5]:
def decoder(conv5,conv4,conv3,conv2,conv1):
    up6 = merge([conv5, conv4], mode='concat', concat_axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    up7 = UpSampling2D((2,2))(conv6)
    up7 = merge([up7, conv3], mode='concat', concat_axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    up8 = UpSampling2D((2,2))(conv7)
    up8 = merge([up8, conv2], mode='concat', concat_axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    up9 = UpSampling2D((2,2))(conv8)
    up9 = merge([up9, conv1], mode='concat', concat_axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)	
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(conv9)
    return decoded

<a id= 'loss'> </a>
## Loss Function

Next you will employ a mean square error in which you'll exclude the values(pixels) of both the ground truth `y_t` and `y_p` that are equal to zero.

In [6]:
def root_mean_sq_GxGy(y_t, y_p):
    a1=1
    zero = tf.constant(0, dtype=tf.float32)
    where = tf.not_equal(y_t, zero)
    a_t=tf.boolean_mask(y_t,where,name='boolean_mask')
    a_p=tf.boolean_mask(y_p,where,name='boolean_mask')
    return a1*(K.sqrt(K.mean((K.square(a_t-a_p)))))

<a id= 'create'> </a>
## Model Definition and Load Weights in all three Autoencoders!

First let's call the `encoder` function by passing in the input to it. Since you are using the merge connection in your architecture the `encoder` function will return the output of five `convolution` layers which you then `merge` with all three decoders output separately.

In [7]:
conv5,conv4,conv3,conv2,conv1 = encoder(input_img)

Now let's create three different models and load the three trained model weights into them.

In [ ]:
autoencoder_1 = Model(input_img, decoder(conv5,conv4,conv3,conv2,conv1))

autoencoder_1.load_weights("../Model/CROSSVAL1/CROSSVAL1_AE1.h5") 

In [ ]:
autoencoder_2 = Model(input_img, decoder(conv5,conv4,conv3,conv2,conv1))

autoencoder_2.load_weights("../Model/CROSSVAL1/CROSSVAL1_AE2.h5")

In [ ]:
autoencoder_3 = Model(input_img, decoder(conv5,conv4,conv3,conv2,conv1))

autoencoder_3.load_weights("../Model/CROSSVAL1/CROSSVAL1_AE3.h5")

<a id= 'test'> </a>
## Model Prediction on Test Volumes: Quantitative and Qualitative Results!

Let's quickly initialise two numpy arrays each of size 11 x 3 x 1 where the first dimension will represent the number of volumes you will use for testing your model; the second dimension will represent the `MSE` and `PSNR` between: Predicted Output 7T MR images and Ground Truth 7T MR images, Predicted Output and Input 3T MR images, Input 3T MR images and Ground Truth 7T MR images; the third dimension will represent the number of channels that you will input to your model!

In [ ]:
mse= np.zeros([11,3,1])
psnr= np.zeros([11,3,1])
i=0 #for iterating over the slices of all the 11 volumes

In the next part of the cell, you will iterate over all 11 volumes one by one. In each iteration you will predict on each volume using all the three autoencoders and finally average out the predictions.<br>
<br>
In each volume you will be iterating over the number of channels of that volume and calculate `PSNR` and `MSE` for three different cases as discussed above.<br>
<br>
Then using `nibabel` library you will save the predicted output, input (3T) and ground truth (7T) as `.nii` format files: each of the 11 volumes comprising of 207 slices.<br>
<br>
Finally, you will save the `PSNR` matrix in a text file using numpy library.

**As stated in the <a href=" https://arxiv.org/abs/1806.06886" target="_blank">paper</a> averaging the predicted outputs helps in reducing noise effects but preserves the local features in the reconstructed images, due to which the PSNR improves over the individual decoder outputs.**

In [ ]:
for j in range(11):
    decoded_imgs_1 = autoencoder_1.predict(ToPredict_images[i:i+207,:,:,:])
    decoded_imgs_2 = autoencoder_2.predict(ToPredict_images[i:i+207,:,:,:])
    decoded_imgs_3 = autoencoder_3.predict(ToPredict_images[i:i+207,:,:,:])
    decoded_imgs = np.mean( np.array([ decoded_imgs_1, decoded_imgs_2,decoded_imgs_3 ]), axis=0 )
    for channel in range(1):
        mse[j,0,channel]=  np.mean((ground_images[i:i+207,:,:,channel] - decoded_imgs[:,:,:,channel]) ** 2)
        psnr[j,0,channel] = 20 * math.log10( 1.0 / math.sqrt(mse[j,0,channel]))
        mse[j,1,channel]=  np.mean((ground_images[i:i+207,:,:,channel] - ToPredict_images[i:i+207,:,:,channel])** 2)
        psnr[j,1,channel] = 20 * math.log10( 1.0 / math.sqrt(mse[j,1,channel]))
        mse[j,2,channel] =  np.mean((ToPredict_images[i:i+207,:,:,channel] - decoded_imgs[:,:,:,channel]) ** 2)
        checklt = math.sqrt(mse[j,2,channel])
        psnr[j,2,channel] = 20 * math.log10( 1.0 / math.sqrt(mse[j,2,channel]))
    obj = nib.Nifti1Image(decoded_imgs, affine)
    string =str(j)+'_crossval1.nii'
    nib.save(obj, save + string)
    obj = nib.Nifti1Image(ground_images[i:i+207,:,:,:], affine)
    string =str(j)+'_ground_images_crossval1.nii'
    nib.save(obj, save + string)
    obj = nib.Nifti1Image(ToPredict_images[i:i+207,:,:,:], affine)
    string =str(j)+'_ToPredict_images_crossval1.nii'
    nib.save(obj, save + string)
    i=i+207


np.savetxt('psnr_all_slices.txt',psnr[:,:,0])